In [ ]:
import pandas as pd
import geopandas as gpd
import os

# Get the current working directory and navigate to data folder
# For Jupyter notebooks, we assume the notebook is in the notebooks/ subdirectory
current_dir = os.getcwd()
# Navigate up one level from notebooks/ to the project root, then to data/
data_dir = os.path.join(os.path.dirname(current_dir), 'data')

# Alternative: if running from project root, use this instead:
# data_dir = os.path.join(current_dir, 'data')

NameError: name '__file__' is not defined

In [ ]:
df_tt = pd.read_csv(os.path.join(data_dir, 'travel_time', 'final_Asymmetric_interzonal_applied_tt_v3.csv'))

In [ ]:
df_jobs = pd.read_csv(os.path.join(data_dir, 'new_job_data', 'jobs_pc5.csv'))  # This file may need to be created/moved

In [ ]:
df_agent = pd.read_csv(os.path.join(data_dir, 'LRO 2023 met ww afstand', 'df_agent_v5.csv'))

In [ ]:
df_wide_tt = pd.read_csv(os.path.join(data_dir, 'travel_time', 'wide_data_tt.csv'))

In [ ]:
df_new_jobs = gpd.read_file(os.path.join(data_dir, 'new_job_data', 'buurten_2022.gpkg'))

The coupling dataset can be exchanged with the one made before.

In [ ]:
df_coupling = pd.read_excel(os.path.join(data_dir, 'geo_coupled', 'koppeling buurten pc4.xlsx'))

To check the urbanity level sorting and meaning

In [ ]:
df_new_jobs[['stedelijkheidsgraad', 'inwoners_totaal']].groupby('stedelijkheidsgraad').mean()

Index(['gwb_code', 'pst_mvp', 'urbanity', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'pst_mvp.1', 'som', 'aantal',
       'average urbanity'],
      dtype='object')

In [8]:
df_coupling[['gwb_code', 'pst_mvp', 'urbanity']]

,gwb_code,pst_mvp,urbanity
0,BU00140000,9712,1.0
1,BU00140001,9711,1.0
2,BU00140002,9711,1.0
3,BU00140003,9718,1.0
4,BU00140004,9717,1.0
...,...,...,...
14312,BU19910603,5411,5.0
14313,BU19910604,5411,5.0
14314,BU19910605,5411,5.0
14315,BU19910606,5411,5.0


In [9]:
df_new_jobs= df_new_jobs.merge(df_coupling[['gwb_code', 'pst_mvp', 'urbanity']], how='left', left_on='zone_id', right_on='gwb_code')

In [10]:
rename_dict = {
    'arbeidsplaatsen_totaal': 'total_jobs',
    'arbeidsplaatsen_woonfunctie': 'residential_jobs',
    'arbeidsplaatsen_standplaats': 'standplace_jobs',
    'arbeidsplaatsen_ligplaats': 'berth_jobs',
    'arbeidsplaatsen_celfunctie': 'cell_function_jobs',
    'arbeidsplaatsen_onderwijsfunctie': 'education_function_jobs',
    'arbeidsplaatsen_industriefunctie': 'industry_function_jobs',
    'arbeidsplaatsen_kantoorfunctie': 'office_function_jobs',
    'arbeidsplaatsen_overige_winkelfunctie': 'other_shop_function_jobs',
    'arbeidsplaatsen_overige_voedselwinkelfunctie': 'other_food_shop_function_jobs',
    'arbeidsplaatsen_supermarkt_voedselwinkelfunctie': 'supermarket_food_shop_function_jobs',
    'arbeidsplaatsen_gezondheidszorgfunctie': 'healthcare_function_jobs',
    'arbeidsplaatsen_sportfunctie': 'sports_function_jobs',
    'arbeidsplaatsen_bijeenkomstfunctie': 'meeting_function_jobs',
    'arbeidsplaatsen_logiesfunctie': 'lodging_function_jobs',
    'arbeidsplaatsen_boerderijfunctie': 'farm_function_jobs',
    'arbeidsplaatsen_terminalfunctie': 'terminal_function_jobs',
    'arbeidsplaatsen_distributiecentrumfunctie': 'distribution_center_function_jobs',
    'arbeidsplaatsen_overige_gebruiksfunctie': 'other_usage_function_jobs',
    'arbeidsplaatsen_gecombineerde_functies': 'combined_function_jobs'
}

df_new_jobs = df_new_jobs.rename(columns=rename_dict)

In [11]:
columns_to_keep = [col for col in df_new_jobs.columns if col.endswith('_jobs')] + ['gwb_code', 'pst_mvp', 'urbanity']
df_new_jobs = df_new_jobs[columns_to_keep]

In [12]:
job_columns = [col for col in df_new_jobs.columns if col.endswith('_jobs')]

# Create aggregation dictionary
agg_dict = {}
for col in job_columns:
    agg_dict[col] = 'sum'
agg_dict['urbanity'] = 'mean'


df_jobs_pc4 = df_new_jobs.groupby('pst_mvp').agg(agg_dict).reset_index()



In [15]:
df_wide_tt.iloc[:, 2:] = df_wide_tt.iloc[:, 2:] / 60


In [13]:
df_jobs_pc4['urbanity'] = df_jobs_pc4['urbanity'].round().astype(int)

In [19]:
df_jobs_pc4['total_jobs'].sum()

9370138.994658967

In [20]:
job_columns = ['pst_mvp', 'total_jobs', 'residential_jobs', 'standplace_jobs',
       'berth_jobs', 'cell_function_jobs', 'education_function_jobs',
       'industry_function_jobs', 'office_function_jobs',
       'other_shop_function_jobs', 'other_food_shop_function_jobs',
       'supermarket_food_shop_function_jobs', 'healthcare_function_jobs',
       'sports_function_jobs', 'meeting_function_jobs',
       'lodging_function_jobs', 'farm_function_jobs', 'terminal_function_jobs',
       'distribution_center_function_jobs', 'other_usage_function_jobs',
       'combined_function_jobs']



import unidecode

def formalize_column(col):
       # Remove accents, replace spaces and special chars with underscores, and lowercase
       col = unidecode.unidecode(col)
       col = col.replace('-', '').replace(',', '').replace(' ', '_').replace('&', 'en')
       col = col.replace('__', '_').replace('(', '').replace(')', '')
       return col.lower()

formal_columns = [formalize_column(col) for col in df_jobs_pc4.columns]
rename_dict = dict(zip(df_jobs_pc4.columns, formal_columns))
df_jobs_pc4 = df_jobs_pc4.rename(columns=rename_dict)

In [ ]:

job_to_sector_group = {
    'residential_jobs': 'work_sector_Remaining',
    'standplace_jobs': 'work_sector_Remaining',
    'berth_jobs': 'work_sector_Remaining',
    'cell_function_jobs': 'work_sector_Remaining',
    'education_function_jobs': 'work_sector_Education',
    'industry_function_jobs': 'work_sector_Industry_Construction',
    'office_function_jobs': 'work_sector_Office_Services',
    'other_shop_function_jobs': 'work_sector_Shop',
    'other_food_shop_function_jobs': 'work_sector_Shop',
    'supermarket_food_shop_function_jobs': 'work_sector_Shop',
    'healthcare_function_jobs': 'work_sector_Care',
    'sports_function_jobs': 'work_sector_Hospitality_Culture_Sports',
    'meeting_function_jobs': 'work_sector_Hospitality_Culture_Sports',
    'lodging_function_jobs': 'work_sector_Hospitality_Culture_Sports',
    'farm_function_jobs': 'work_sector_Remaining',
    'terminal_function_jobs': 'work_sector_Remaining',
    'distribution_center_function_jobs': 'work_sector_Remaining',
    'other_usage_function_jobs': 'work_sector_Remaining',
    'combined_function_jobs': 'work_sector_Remaining',
}

unique_sectors = list(set(job_to_sector_group.values()))

for sector in unique_sectors:
    cols = [col for col, grp in job_to_sector_group.items() if grp == sector]
    print(cols, sector)
    df_jobs_pc4[f'{sector}_jobs'] = df_jobs_pc4[cols].sum(axis=1)



['industry_function_jobs'] work_sector_Industry_Construction
['other_shop_function_jobs', 'other_food_shop_function_jobs', 'supermarket_food_shop_function_jobs'] work_sector_Shop
['sports_function_jobs', 'meeting_function_jobs', 'lodging_function_jobs'] work_sector_Hospitality_Culture_Sports
['education_function_jobs'] work_sector_Education
['residential_jobs', 'standplace_jobs', 'berth_jobs', 'cell_function_jobs', 'farm_function_jobs', 'terminal_function_jobs', 'distribution_center_function_jobs', 'other_usage_function_jobs', 'combined_function_jobs'] work_sector_Remaining
['healthcare_function_jobs'] work_sector_Care
['office_function_jobs'] work_sector_Office_Services


In [23]:
job_columns = [col for col in df_jobs_pc4.columns if col.endswith('_jobs')]

In [24]:
work_sector_columns = [col for col in job_columns if col.startswith('work_sector_')] + ['total_jobs', 'urbanity']

In [ ]:
# removing invalid codes
df_jobs['pc4_code'] = pd.to_numeric(df_jobs['pc4_code'], errors='coerce').dropna().astype(int)

df_jobs_pc4['pst_mvp'] = pd.to_numeric(df_jobs_pc4['pst_mvp'], errors='coerce').dropna().astype(int)
df_jobs_pc4 = df_jobs_pc4[df_jobs_pc4['pst_mvp'].isin(df_jobs['pc4_code'].unique())]

In [ ]:
# Creating new column names for each pc4_code
new_columns = {}
for _, row in df_jobs_pc4.iterrows():
    pc4_code = int(row['pst_mvp']) if not pd.isna(row['pst_mvp']) else None
    for sector in work_sector_columns:
        new_col_name = f"{sector}_{pc4_code}"
        new_columns[new_col_name] = row[sector]

# a new dataframe
df_jobs_wide = pd.DataFrame([new_columns])
df_jobs_wide

,work_sector_Industry_Construction_jobs_1011,work_sector_Shop_jobs_1011,work_sector_Hospitality_Culture_Sports_jobs_1011,work_sector_Education_jobs_1011,work_sector_Remaining_jobs_1011,work_sector_Care_jobs_1011,work_sector_Office_Services_jobs_1011,total_jobs_1011,urbanity_1011,work_sector_Industry_Construction_jobs_1012,...,urbanity_9995,work_sector_Industry_Construction_jobs_9997,work_sector_Shop_jobs_9997,work_sector_Hospitality_Culture_Sports_jobs_9997,work_sector_Education_jobs_9997,work_sector_Remaining_jobs_9997,work_sector_Care_jobs_9997,work_sector_Office_Services_jobs_9997,total_jobs_9997,urbanity_9997
0,776.784091,892.4406,2460.245597,706.632673,3796.112586,48.083003,6364.772002,15045.070551,1.0,723.007734,...,5.0,40.029481,0.0,28.901491,8.089959,12.210298,0.0,0.0,89.231228,5.0


In [28]:
df_wide_tt.rename(columns=lambda col: f'tt_{col}' if col not in ['id_nr', 'living_adress'] else col, inplace=True)

In [29]:

df_jobs_wide = pd.concat([df_jobs_wide]*2550, ignore_index=True)

In [30]:
df_combined = pd.concat([df_wide_tt, df_jobs_wide ], axis=1)

In [31]:
formal_columns = [formalize_column(col) for col in df_agent.columns]
rename_dict = dict(zip(df_agent.columns, formal_columns))
df_agent = df_agent.rename(columns=rename_dict)

Grouping household variable

In [33]:
HH_compound_map = {
    'child_<12': 'with_child',
    'child_>12': 'with_child',
    'other_people': 'without_child',
    'without_child': 'without_child', 
    'alone': 'without_child',
    'unknown': 'without_child',
}


df_agent['hh_compund_grouped'] = df_agent['hh_compound'].map(HH_compound_map)


This function makes a dummy variable (one column for each category level).

In [34]:
def get_dummied(df, columns, prefix='', prefix_sep=''):
    """
    Returns a DataFrame with dummy variables for the specified columns.
    
    Args:
        df (pd.DataFrame): Input DataFrame.
        columns (list): List of column names to dummy.
        prefix (str): Prefix for new columns.
        prefix_sep (str): Separator between prefix and column name.
        
    Returns:
        pd.DataFrame: DataFrame with dummy variables.
    """
    return pd.get_dummies(df, columns=columns, prefix=prefix, prefix_sep=prefix_sep)

Mapping LRO work sector to CBS work sector groups.

In [35]:

sector_mapping = {
    1: 'Agriculture_and_fisheries',
    2: 'Industry',
    3: 'Construction',
    4: 'Utilities',
    5: 'Commerce',
    6: 'Hospitality',
    7: 'Transportation',
    8: 'ICT',
    9: 'Financial_services',
    10: 'Business_services',
    11: 'Public_Administration',
    12: 'Education',
    13: 'Health_and_welfare',
    14: 'Culture',
    15: 'Other_services'
}

df_agent['work_sector_name'] = df_agent['work_sector'].map(sector_mapping)
sector_group_map = {
    'Agriculture_and_fisheries': 'Remaining',
    'Industry': 'Industry_Construction',
    'Construction': 'Industry_Construction',
    'Utilities': 'Remaining',
    'Commerce': 'Shop',
    'Hospitality': 'Hospitality_Culture_Sports',
    'Transportation': 'Industry_Construction',
    'ICT': 'Office_Services',
    'Financial_services': 'Office_Services',
    'Business_services': 'Office_Services',
    'Public_Administration': 'Office_Services',
    'Education': 'Education',
    'Health_and_welfare': 'Care',
    'Culture': 'Hospitality_Culture_Sports',
    'Other_services': 'Remaining'
}

df_agent['sector_cluster'] = df_agent['work_sector_name'].map(sector_group_map)

Grouping age categories

In [36]:
df_agent['age_category_BC'] = df_agent['age_category'].isin(['B', 'C'])
df_agent['age_category_DE'] = df_agent['age_category'].isin(['D', 'E'])

In [37]:
agent_char = [[
       'gender', 'work_flexibility',
       'sector_cluster', 'hh_compund_grouped', 'age_category']]

In [39]:
df_agent['gender'].unique()

array([ 1.,  2., nan])

In [40]:

dummy_vars = agent_char[0]

df_agent_dummies = get_dummied(df_agent, columns=dummy_vars, prefix=dummy_vars, prefix_sep='_')

In [41]:
dummy_columns = [col for col in df_agent_dummies.columns if any(var in col for var in dummy_vars)]

# Keep only the specified columns plus all dummy columns
columns_to_keep = ['id_nr', 'destination_pc4'] + dummy_columns

df_agent_filtered = df_agent_dummies[columns_to_keep]

In [42]:
df_wide = df_agent_filtered.merge(df_combined, on='id_nr', how='left')

In [46]:
unique_pc4 = df_tt['pc4_from'].unique()

In [ ]:

unique_pc4 = df_tt['pc4_from'].unique()

df_tt['afstand'] = df_tt['afstand'] * 2 # 2 ways

df_distance_wide = df_tt.pivot_table(
    index='pc4_from', 
    columns='pc4_to', 
    values='afstand', 
    fill_value=0
)

df_distance_wide = df_distance_wide.reset_index()

# Make distance columns
distance_columns = ['living_adress'] + ['afstand_' + str(col) for col in df_distance_wide.columns[1:]]
df_distance_wide.columns = distance_columns

# Merging with existing dataset
df_wide_with_distance = df_wide.merge(df_distance_wide, on='living_adress', how='left')

In [49]:
df_wide_with_distance.merge(df_wide_tt, how='left', on= 'id_nr')

,id_nr,destination_pc4,age_category_BC,age_category_DE,gender_1.0,gender_2.0,work_flexibility_flexible,work_flexibility_non_flexible,sector_cluster_Care,sector_cluster_Education,...,tt_6681_y,tt_6176_y,tt_4465_y,tt_7611_y,tt_1437_y,tt_4333_y,tt_7314_y,tt_7815_y,tt_6882_y,tt_1617_y
0,1.0,1011.0,False,True,True,False,True,False,False,False,...,161.900896,265.343930,256.999794,199.703138,52.282505,265.863832,130.200831,238.654433,148.981497,75.667285
1,47.0,1217.0,True,False,True,False,False,True,False,False,...,163.266074,271.069417,260.736811,198.235912,53.639542,267.641693,128.733606,237.187210,147.514271,71.669864
2,52.0,1034.0,True,False,False,True,True,False,True,False,...,156.501056,264.304401,256.066017,191.470896,51.123459,264.650088,121.968590,230.422192,140.749254,70.404207
3,53.0,3607.0,True,False,True,False,False,True,False,True,...,156.501056,264.304401,256.066017,191.470896,51.123459,264.650088,121.968590,230.422192,140.749254,70.404207
4,57.0,1061.0,True,False,True,False,False,True,True,False,...,156.501056,264.304401,256.066017,191.470896,51.123459,264.650088,121.968590,230.422192,140.749254,70.404207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2545,15023.0,4416.0,False,True,True,False,False,True,False,False,...,243.375486,288.082764,14.653277,356.436781,232.725431,45.228004,296.288377,406.102897,266.811778,306.376955
2546,15032.0,1042.0,False,True,True,False,False,True,False,False,...,165.510571,268.809922,243.204683,208.929682,36.371541,250.109569,139.427374,247.880975,156.613302,69.119931
2547,15035.0,3901.0,False,True,True,False,False,True,False,False,...,69.142219,217.968768,239.158401,141.006050,118.501100,257.973776,80.427825,204.006475,53.463796,170.206990
2548,15041.0,1315.0,True,False,False,True,True,False,False,False,...,180.994244,288.476839,263.449824,215.964083,56.352560,270.354712,146.461776,254.915381,165.242442,46.473843


Checking if 2-way distances are reasonable

In [ ]:

sample_distances = df_tt['afstand'].dropna()
print(f"Current distances (after *2 for 2-way):")
print(f"  Mean: {sample_distances.mean():.0f} meters ({sample_distances.mean()/1000:.1f} km)")
print(f"  Median: {sample_distances.median():.0f} meters ({sample_distances.median()/1000:.1f} km)")
print(f"  Min: {sample_distances.min():.0f} meters ({sample_distances.min()/1000:.1f} km)")
print(f"  Max: {sample_distances.max():.0f} meters ({sample_distances.max()/1000:.1f} km)")



Checking if 2-way distances are reasonable...
Current distances (after *2 for 2-way):
  Mean: 289547 meters (289.5 km)
  Median: 276842 meters (276.8 km)
  Min: 0 meters (0.0 km)
  Max: 864373 meters (864.4 km)


In [ ]:
import numpy as np

print("Checking travel time vs distance relationship...")

# Get a sample of matching travel time and distance data
sample_zones = ['1011', '1217', '1034', '1042', '1061']  

for zone in sample_zones[:3]:  # Check first 3 zones
    tt_col = f'tt_{zone}'
    dist_col = f'afstand_{zone}'
    
    if tt_col in df_wide_with_distance.columns and dist_col in df_wide_with_distance.columns:
        mask = (df_wide_with_distance[tt_col].notna()) & \
               (df_wide_with_distance[dist_col].notna()) & \
               (df_wide_with_distance[tt_col] > 0) & \
               (df_wide_with_distance[dist_col] > 0)
        
        if mask.sum() > 0:
            tt_values = df_wide_with_distance.loc[mask, tt_col]  # minutes
            dist_values = df_wide_with_distance.loc[mask, dist_col]  # meters
            
            implied_speed_kmh = (dist_values / 1000) / (tt_values / 60)  # km/h
            
            print(f"\nZone {zone} (sample size: {len(tt_values)}):")
            print(f"  Travel time: {tt_values.mean():.1f} min (range: {tt_values.min():.1f}-{tt_values.max():.1f})")
            print(f"  Distance: {dist_values.mean()/1000:.1f} km (range: {dist_values.min()/1000:.1f}-{dist_values.max()/1000:.1f})")
            print(f"  Implied speed: {implied_speed_kmh.mean():.1f} km/h (range: {implied_speed_kmh.min():.1f}-{implied_speed_kmh.max():.1f})")
            
            reasonable_speed = (implied_speed_kmh >= 15) & (implied_speed_kmh <= 100)
            print(f"  Reasonable speeds (15-100 km/h): {reasonable_speed.sum()}/{len(implied_speed_kmh)} ({100*reasonable_speed.mean():.1f}%)")

print(f"\n=== SUMMARY ===")
print("Expected car speeds for different scenarios:")
print("  - City driving: 20-50 km/h")
print("  - Highway driving: 60-120 km/h") 
print("  - Mixed (realistic average): 30-80 km/h")
print("  - Round-trip work commute: 25-70 km/h")

# Quick check 
all_distances = df_wide_with_distance[[col for col in df_wide_with_distance.columns if col.startswith('afstand_')]].values.flatten()
all_distances = all_distances[~pd.isna(all_distances)]
all_distances = all_distances[all_distances > 0]

print(f"\nAll distance statistics:")
print(f"  Mean: {all_distances.mean()/1000:.1f} km")
print(f"  Median: {np.median(all_distances)/1000:.1f} km") 
print(f"  95th percentile: {np.percentile(all_distances, 95)/1000:.1f} km")
print(f"  Max: {all_distances.max()/1000:.1f} km")

Checking travel time vs distance relationship...

Zone 1011 (sample size: 2537):
  Travel time: 161.7 min (range: 6.5-441.3)
  Distance: 212.3 km (range: 2.1-469.0)
  Implied speed: 75.2 km/h (range: 19.2-96.8)
  Reasonable speeds (15-100 km/h): 2537/2537 (100.0%)

Zone 1217 (sample size: 2537):
  Travel time: 150.5 min (range: 6.3-456.5)
  Distance: 192.9 km (range: 3.5-440.8)
  Implied speed: 73.8 km/h (range: 25.4-94.8)
  Reasonable speeds (15-100 km/h): 2537/2537 (100.0%)

Zone 1034 (sample size: 2537):
  Travel time: 159.7 min (range: 4.9-430.3)
  Distance: 219.2 km (range: 1.7-479.8)
  Implied speed: 79.4 km/h (range: 20.7-98.7)
  Reasonable speeds (15-100 km/h): 2537/2537 (100.0%)

=== SUMMARY ===
Expected car speeds for different scenarios:
  - City driving: 20-50 km/h
  - Highway driving: 60-120 km/h
  - Mixed (realistic average): 30-80 km/h
  - Round-trip work commute: 25-70 km/h

All distance statistics:
  Mean: 280.1 km
  Median: 267.2 km
  95th percentile: 531.4 km
  Max: 

In [53]:
df_test = df_wide_with_distance.copy()

Filtering observations with relevant jobs in chosen zones

In [ ]:

initial_count = len(df_test)

work_sectors = ['Care', 'Education', 'Hospitality_Culture_Sports', 
                'Industry_Construction', 'Office_Services', 'Remaining', 'Shop']

# Create filter for observations with relevant jobs in chosen zones
valid_observations = []

for idx, row in df_test.iterrows():
    chosen_zone = int(row['destination_pc4'])
    keep_observation = False
    
    # Check if person has jobs in their sector in the chosen zone
    for sector in work_sectors:
        sector_var = f'sector_cluster_{sector}'
        job_var = f'work_sector_{sector}_jobs_{chosen_zone}'
        
        # If person works in this sector AND chosen zone has jobs in this sector
        if (sector_var in df_test.columns and 
            job_var in df_test.columns and
            row[sector_var] == 1 and 
            row[job_var] > 0):
            keep_observation = True
            break
    
    valid_observations.append(keep_observation)

# Filter the dataframe
df_test = df_test[valid_observations].copy()
final_count = len(df_test)
removed_count = initial_count - final_count

print(f"   - Removed {removed_count} observations with no relevant jobs in chosen zones")
print(f"   - Remaining observations: {final_count} (from {initial_count})")


5.5. Filtering observations with relevant jobs in chosen zones...
   - Removed 71 observations with no relevant jobs in chosen zones
   - Remaining observations: 2479 (from 2550)


Saving the files in seperate files - one for extended model and one for basic model.

In [ ]:
df_test.to_csv(os.path.join(data_dir, 'final_wide_format', 'wide_data_filtered_v6.csv'), index=False)

In [ ]:
df_wide_with_distance.to_csv(os.path.join(data_dir, 'final_wide_format', 'wide_data_UNfiltered_v6.csv'), index=False)